# NaijaLingo TTS SDK Tests

This notebook exercises all TTS pipelines via the Speech Service:
- Finetuned (NaijaLingo finetuned models)
- Open-source MMS TTS (Facebook)
- Gemini TTS (requires GEMINI_API_KEY configured on the service)



In [1]:
import sys
sys.path.insert(0, "/home/chukypedro15/Naijalingo/NaijaLingoKit/src")

from naijalingo_tts import SpeechServiceTTSClient, SpeechServiceConfig, suppress_all_logs

# Configure base URL if needed (defaults to http://localhost:8000)
BASE_URL = "http://localhost:8000"

suppress_all_logs()
client = SpeechServiceTTSClient(SpeechServiceConfig(base_url=BASE_URL))
print("Service URL:", client.config.base_url)


Service URL: http://localhost:8000


In [6]:
# Helper to download generated files from the speech service
import requests
from pathlib import Path

def download_tts_file(file_path: str, base_url: str = BASE_URL, local_dir: str = "downloads") -> Path:
    filename = Path(file_path).name
    url = f"{base_url.rstrip('/')}/api/tts/download/{filename}"
    out_dir = Path(local_dir)
    out_dir.mkdir(parents=True, exist_ok=True)
    local_path = out_dir / filename
    r = requests.get(url, timeout=180)
    r.raise_for_status()
    with open(local_path, "wb") as f:
        f.write(r.content)
    print("Downloaded:", local_path)
    return local_path


## Finetuned TTS: voices and models


In [2]:
voices = client.get_finetuned_voices()
models = client.get_finetuned_models()
voices, models


({'speakers': ['Aliyu',
   'Talatu',
   'Bashir',
   'Asabe',
   'Adamu',
   'Jumai',
   'Yakub',
   'A’isha',
   'Rahmah',
   'Kyauta',
   'Aminah',
   'Yusuf',
   'Hauwa',
   'Oyewale',
   'Kehinde',
   'Oladipo',
   'Bolanle',
   'Adedamola',
   'Abiola',
   'Ifeoluwa',
   'Abiodun',
   'Taiwo',
   'Ayomide',
   'Adebowale',
   'Bankole',
   'Monife',
   'Ademola',
   'Femi',
   'Adunni',
   'Damilola',
   'Lolade',
   'Adaeze',
   'Amarachi',
   'Uchechi',
   'Obinna',
   'Adaugo',
   'Ifeoma',
   'Joy',
   'Samuel',
   'Josphine',
   'Kate',
   'Henry',
   'Simon',
   'Anita',
   'Ibrahim',
   'Musa',
   'Sandra']},
 {'hausa': 'chukypedro/hausa-tts-400m-0.3-pt',
  'igbo': 'chukypedro/igbo-tts-400m-0.3-pt',
  'yoruba': 'chukypedro/yoruba-tts-400m-0.3-pt',
  'pidgin': 'chukypedro/pidgin-tts-400m-0.3-pt'})

## Finetuned TTS: synthesize one file


In [ ]:
resp_ft = client.synthesize_finetuned(
    text="Sannu, ya ya kake?",
    model_name="chukypedro/hausa-tts-400m-0.3-pt",
    speaker_id="Aliyu",
    device="cpu",
    output_filename="tts_naijalingo_sample.wav",
)
resp_ft


In [ ]:
# Download the finetuned output from the service and play locally
from IPython.display import Audio as APlay
import soundfile as sf

local_ft = download_tts_file(resp_ft["file_path"], base_url=BASE_URL)
audio, sr = sf.read(str(local_ft))
APlay(audio, rate=sr)


## Open-source MMS TTS: synthesize


In [9]:
# Yoruba example
yoruba_resp_mms = client.synthesize_open_source(
    text="Ẹ káàrọ̀ o, báwo ni ìlera rẹ̀?",
    language="yoruba",   # 'hausa' | 'yoruba' | 'pidgin'
    device="cpu",
    output_filename="tts_mms_yoruba.wav",
)
print(yoruba_resp_mms)
print("--------------------------------")

# Hausa example
hausa_resp_mms = client.synthesize_open_source(
    text="Ina kwana, lafiya kalau?",
    language="hausa",   # 'hausa' | 'yoruba' | 'pidgin'
    device="cpu",
    output_filename="tts_mms_hausa.wav",
)
print(hausa_resp_mms)
print("--------------------------------")

# Pidgin English example
pidgin_resp_mms = client.synthesize_open_source(
    text="Good morning, how your body dey?",
    language="pidgin",   # 'hausa' | 'yoruba' | 'pidgin'
    device="cpu",
    output_filename="tts_mms_pidgin.wav",
)
print(pidgin_resp_mms)
print("--------------------------------")



{'file_path': 'tts_result/tts_mms_yoruba.wav', 'language': 'yoruba', 'device': 'cpu'}
--------------------------------
{'file_path': 'tts_result/tts_mms_hausa.wav', 'language': 'hausa', 'device': 'cpu'}
--------------------------------
{'file_path': 'tts_result/tts_mms_pidgin.wav', 'language': 'pidgin', 'device': 'cpu'}
--------------------------------


In [10]:
# Download the MMS output from the service and play locally
from IPython.display import Audio as APlay
import soundfile as sf

# Yoruba example
yoruba_local_mms = download_tts_file(yoruba_resp_mms["file_path"], base_url=BASE_URL)
audio, sr = sf.read(str(yoruba_local_mms))
APlay(audio, rate=sr)

# Hausa example
hausa_local_mms = download_tts_file(hausa_resp_mms["file_path"], base_url=BASE_URL)
audio, sr = sf.read(str(hausa_local_mms))
hausa_local_mms

# Pidgin English example
pidgin_local_mms = download_tts_file(pidgin_resp_mms["file_path"], base_url=BASE_URL)
audio, sr = sf.read(str(pidgin_local_mms))
pidgin_local_mms



Downloaded: downloads/tts_mms_yoruba.wav
Downloaded: downloads/tts_mms_hausa.wav
Downloaded: downloads/tts_mms_pidgin.wav


PosixPath('downloads/tts_mms_pidgin.wav')

## Gemini TTS: synthesize (requires GEMINI_API_KEY on service)


In [11]:
try:
    resp_g = client.synthesize_gemini(
        text="How are you today?",
        language="pidgin",      # 'yoruba' | 'igbo' | 'hausa' | 'pidgin'
        voice_name="Zephyr",
        style_prompt=None,
        output_filename="tts_gemini_pidgin.wav",
    )
    resp_g
except Exception as e:
    print("Gemini call failed:", e)
    print("Ensure GEMINI_API_KEY is set on the speech service.")


In [12]:
# Pidgin English example
pidgin_local_google = download_tts_file(resp_g["file_path"], base_url=BASE_URL)
audio, sr = sf.read(str(pidgin_local_google))
pidgin_local_google

Downloaded: downloads/tts_gemini_pidgin.wav


PosixPath('downloads/tts_gemini_pidgin.wav')